In [9]:
import pandas as pd

In [10]:
df=pd.read_excel("/Users/santhanakrishnans/Downloads/ford_service_review_processed.xlsx")
data=df.dropna(subset=['review'])
data["review_original"]=data["review"]
data=data.reset_index()

/var/folders/gl/57089jm950v5k1zmmrkghv4r0000gn/T/ipykernel_2383/510883715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["review_original"]=data["review"]


In [11]:
len(df["review"])

2092

In [12]:
#data=data.head(30)

In [13]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [14]:

import os

# Get the number of logical cores
num_cores = os.cpu_count()




In [15]:
num_cores

8

In [16]:
import pandas as pd
from tqdm import tqdm
from ollama import chat
import re
from concurrent.futures import ThreadPoolExecutor

PREDEFINED_TOPIC_STYLE = """
The topics you identify should follow this pattern: 
1. Use clear, concise phrases that reflect a broader category (e.g., "Product Quality" instead of "Quality of product").
2. Avoid overly specific or long phrases.
3. Use two to three words for each topic, aiming to summarize the key aspects of the review in a short, general way.

Examples of appropriate topics:
- Product Quality
- Staff 
- Service Time
- Music System
- Recall
- Brake
- Oil Change/Refill
- Wheel/Tires
- Product Quality
- Service Cost
- Service Diagnostics
- Facilities
- Windshield
- Door
- Battery
- Engine
-Product Availability
"""
# Define the review analysis prompt with a strict output format
PROMPT_TEMPLATE = '''

Please analyze the following review and identify the most relevant topics discussed. 
The following is a predefined list of possible topics that you should choose from:
- Product Quality
- Staff
- Service Time
- Music System
- Recall
- Brake
- Oil Change/Refill
- Wheel/Tires
- Product Quality
- Service Cost
- Service Diagnostics
- Facilities
- Windshield
- Door
- Battery
- Engine
- Gear Box
- Charging
- Product Availability
- Not Classified


You should **only** choose topics from this list, based on the review. If multiple topics apply, list them in order of relevance (from most relevant to least relevant).
If You are not able to find relevant topic assign it as **Not Classified**

Below are **few-shot examples** showing how to identify the topics :

Example 1:
Review: "The music feature stopped working again. I'm really frustrated with this issue."
- Chain of Thought:
  1. The review mentions "music feature", which suggests reviewer talks about Music System. 
  2.Stopped working again suggest a Product failure hence Product Quality
- Key topics: Music System,Product Quality


Example 2:
Review: "The staff at the dealership were amazing! Tomasina provided great service, and I had a smooth experience."
- Chain of Thought:
  1. The review highlights positive experiences with the staff or service person hence "Staff"
- Key topics: Staff 


Example 3:
Review: "Came to change brake pad. The product doesn't meet the standards, and I'm disappointed."
- Chain of Thought:
  1. The review mentions "brake pad", indicating the issue is with a specific part.
  2. The phrase "doesn't meet the standards" suggests a problem with the product’s quality.
- Key topics: Brake,Product Quality


Example 4:
Review: "I had a quick and easy service appointment. The staff were courteous, and the mechanic did a great job in quick time."
- Chain of Thought:
  1. The review mentions a "quick and easy service appointment", which suggests that the service time was favorable.
  2. The staff were "courteous", indicating a positive review of staff or service person behavior.
  3. The mechanic did a "great job", suggesting that service quality was also good and broadly classify it into staff.
- Key topics: Staff,Service Time


Example 5:
Review: "While the service was good, the facilities were basic and not ideal. I wish there were more amenities."
- Chain of Thought:
  1. The review mentions a service was good, which suggest overall experience was good
  2. however they also mention facilities were basic and not ideal and they would like more aminities hence indicating negative sentiment due to lack of Facilities
- Key topics: Service quality, Facilities


Example 6:
Review: "I found only one screw on my left front wheel after recall by this service center. That mistake totally ruined my wheel and cost hundreds of dollars to fix."
- Chain of Thought:
  1. The review mentions only one screw was there,which means overall Service quality was bad
  2. Customer also mentions a specific part as wheel hence identify Wheel/Tires,also this was a recall service hence labelling recall
- Key topics: Wheel/Tires, Recall,Service Quality


Example 7:
Review: "I went to get an inspection and they told me to fix a bulb for $400. When I did it outside, they did it for $49. They were ripping people off. I waited 3 hours and they still couldn’t figure out what was wrong with the car."
- Chain of Thought:
  1. The review mentions feeling ripped off which indicates high Service Cost
  2. Review also takes about waiting for 3 hours hence Service Time
  3. It also mentions that they could not figure out whats wrong with thw car hence Service Diagnostics
- Key topics: Service Cost,Service Diagnostics,Service Time


Example 8:
Review: "Worst customer service ever. Took my F150 for a recall. From the start, she was so negative about the issue, told me it would take months to fix, and that I would have to leave the truck there. She is not even a technician and gave a diagnostic before inspecting the car. I was in disbelief. I suggest taking your Ford to a better service department."
 1. The Customer takes about a service person being negative and expresses disappointment also they have mentioned it was  for a recall
 2.Customer has also expressed that Staff mentioned it would take months to fix hence a Topic Service Time and also alleges wrong diagnosis hence Service Diagnostics
- Key topics: Staff ,Recall,Service Time,Service Diagnostics




Each output **must strictly follow the format below** ,Always **Output**Key topics andSentiment  **Do Not Output** Chain of Thought or Other Note:

- Key topics:[Topic1, Topic2, Topic3]
- Sentiment:[Positive/Negative/Neutral].


Now, analyze the following review:
{review}
'''

# Function to send a review for analysis
def analyze_review(review: str, model: str = "llama3.2", max_retries: int = 3) -> str:
    """
    Analyzes a review using the LLM model.

    Args:
    review (str): The review text to analyze.
    model (str, optional): The LLM model to use. Defaults to "llama3.2".
    max_retries (int, optional): The maximum number of retries. Defaults to 3.

    Returns:
    str: The raw LLM response.
    """
    prompt = PROMPT_TEMPLATE.format(review=review)
    messages = [{'role': 'user', 'content': prompt}]

    for attempt in range(max_retries):
        try:
            # Use a more specific exception type instead of bare Exception
            response = chat(model, messages=messages)
            return response.message.content.strip()
        except RuntimeError as e:
            # Provide more informative error messages
            print(f"Retry {attempt + 1}/{max_retries} failed: {e}")
        except Exception as e:
            # Catch any other unexpected exceptions
            print(f"Unexpected error: {e}")
            break

    # Raise a more specific exception instead of RuntimeError
    raise TimeoutError("Failed to get a valid response from the LLM after retries.")


    
# Function to process a single review and parse the result
import re

#Here's an updated version of the process_review function to handle this new Raw LLM Output:
#Python
#import re

import re

def process_review(review: str) -> dict:
    """
    Processes a review and extracts the topics and sentiment.

    Args:
    review (str): The review text to process.

    Returns:
    dict: A dictionary containing the extracted topics and sentiment.
    """
    try:
        # Get raw LLM response
        review_output = analyze_review(review)
        print(f"Raw LLM Output for review: {review}\n{review_output}\n")

        # Extract key topics
        key_topics_match = re.search(r"- Key topics:(.*?)\n- Sentiment:", review_output, re.DOTALL)
        if key_topics_match:
            key_topics = [topic.strip() for topic in key_topics_match.group(1).split("\n- ")]
        else:
            key_topics_match = re.search(r"-Key topics:\s*(.*?)\s*-Sentiment:", review_output, re.DOTALL)
            if key_topics_match:
                key_topics = [topic.strip() for topic in key_topics_match.group(1).split("\n")]
            else:
                key_topics_match = re.search(r"- Key topics: (.*?)$", review_output, re.DOTALL)
                if key_topics_match:
                    key_topics = [topic.strip() for topic in key_topics_match.group(1).split(",")]
                else:
                    key_topics_match = re.search(r"Key topics:\s*(.*?)\s*Sentiment:", review_output, re.DOTALL)
                    if key_topics_match:
                        key_topics = [topic.strip() for topic in key_topics_match.group(1).split("\n- ")]
                    else:
                        key_topics = ["Unknown"]

        # Extract sentiment
        sentiment_match = re.search(r"- Sentiment: (.*?)$", review_output, re.IGNORECASE)
        if sentiment_match:
            sentiment = sentiment_match.group(1).strip().capitalize()
        else:
            sentiment_match = re.search(r"-Sentiment: (.*?)$", review_output, re.IGNORECASE)
            if sentiment_match:
                sentiment = sentiment_match.group(1).strip().capitalize()
            else:
                sentiment = "Unknown"

        return {"topics": key_topics, "sentiment": sentiment}

    except Exception as e:
        # Catch any unexpected exceptions
        print(f"Unexpected error processing review: {e}")
        return {"topics": ["Unknown"], "sentiment": "Unknown"}
        
def parallel_processing(reviews, max_workers=5):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(
            executor.map(process_review, reviews),
            total=len(reviews),
            desc="Processing Reviews"
        ))
    return results

# Process reviews in parallel
results = parallel_processing(data['review'])

# Align the results with the DataFrame
data['topics'] = [result['topics'] for result in results]
data['sentiment'] = [result['sentiment'] for result in results]

# Display the updated DataFrame



Processing Reviews:   0%|                              | 0/1181 [00:00<?, ?it/s]

Raw LLM Output for review: I had a very bad experience in that location. They don’t listen to you and they have bad attitude.Just saying come after one or two weeks every time. They take my car to fix. After two days they called me and said take my car and come back again after two weeks. I don’t recommend them.
- Key topics:[Staff, Facilities]
- Sentiment:Negative

Raw LLM Output for review: Lies and condescending attitudes from the service rep and their manager. Take your business elsewhere. I called to make an appointment to service my F150 Raptor, and gave a detailed explanation  to the rep that my sunroof is leaking, need an oil change, and inspection and door lock fixed. When I arrived Juan the service rep say they have no record of anything except oil change and inspection and they don’t fix leaks. It a ford product who does then? I asked that and then he say I’m saying this a second time don’t make me have to say it a third time We Don’t Fix Leaks! Then he takes me to the manag

Processing Reviews:   0%|                    | 1/1181 [00:27<9:01:38, 27.54s/it]

Raw LLM Output for review: Took my truck in to get the front brakes replaced. A week later, my brake fluid reservoir busted and my brakes failed. In my discussions with other mechanics there was a high probability this could have been caused by the brake replacement. Ford Lincoln took no responsibility and made me pay the full cost to repair everything.
This is terrible customer service and NOT TRUSTWORTHY.
- Key topics:[Brake,Product Quality]
- Sentiment:[Negative]

Raw LLM Output for review: The service department encountered challenges in diagnosing the charging system fault.

The technician suggested that the battery failure led to the P0620 code, but additional investigation revealed damage to the voltage regulator in the alternator.

The service fee of over $150 for a brief job, encompassing code reading and battery voltage testing, which I CAN DO AT HOME, seems disproportionately high.

It's infuriating that they not only failed to fix the problem but also couldn't diagnose it p

Processing Reviews:   0%|                    | 3/1181 [00:30<2:39:29,  8.12s/it]

...

Raw LLM Output for review: Fast service
- Key topics:Service Time
- Sentiment: [Positive]

Raw LLM Output for review: Interesting
- Key topics:Not Classified
- Sentiment:Not Classified

Raw LLM Output for review: I know very little about this place but they say that what they sell is very good, they are the cars and I don't know the address so little.
- Key topics:[Not Classified]
- Sentiment:[Neutral]

Raw LLM Output for review: Too busy
- Key topics: [Not Classified]
- Sentiment: [Negative]



Processing Reviews: 100%|██████████████████▉| 1179/1181 [17:50<00:02,  1.04s/it]

Raw LLM Output for review: A beautiful showroom where you can see the new Cybertruck and Tesla 3, and Tesla's new robot is also here.
- Key topics:[Not Classified, Not Classified]
-Sentiment: [Neutral]



Processing Reviews: 100%|███████████████████| 1181/1181 [17:53<00:00,  1.10it/s]

Raw LLM Output for review: Car cool
- Key topics: [Not Classified,Not Classified,Not Classified,Not Classified,Not Classified,Not Classified,Not Classified,Not Classified,Not Classified]
- Sentiment: [Negative]



In [21]:
results

[{'topics': ['[Service Diagnostics, Service Cost]'], 'sentiment': 'Unknown'},
 {'topics': ['[Product Quality, Service Diagnostics]'],
  'sentiment': 'Unknown'},
 {'topics': ['[Staff, Facilities]'], 'sentiment': 'Unknown'},
 {'topics': ['[Staff, Facilities]'], 'sentiment': 'Unknown'},
 {'topics': ['[Brake,Product Quality]'], 'sentiment': 'Unknown'},
 {'topics': ['[Biener Ford, Service Quality]'], 'sentiment': 'Unknown'},
 {'topics': ['[Door, Recall, Staff]'], 'sentiment': 'Negative'},
 {'topics': ['[Service Time,Staff,SERVICE COST]\n-'], 'sentiment': 'Positive'},
 {'topics': ['[Staff]\n-'], 'sentiment': 'Unknown'},
 {'topics': ['[Service Quality, Door]'], 'sentiment': 'Unknown'},
 {'topics': ['[Staff, Service Cost]'], 'sentiment': 'Negative'},
 {'topics': ['Staff'], 'sentiment': 'Unknown'},
 {'topics': ['[Service Time, Facilities]'], 'sentiment': 'Unknown'},
 {'topics': ['[Service Time, Service Cost, Staff]'], 'sentiment': 'Unknown'},
 {'topics': ['[Staff,Service Time]'], 'sentiment': '

In [18]:
data.columns

Index(['index', 'serviceCentre', 'dummy', 'overall rating', 'dummy1',
       'geo_location', 'Unnamed: 5', 'Unnamed: 6', 'user', 'Unnamed: 8',
       'date', 'rating', 'review', 'Unnamed: 12', 'response',
       'review_original', 'topics', 'sentiment'],
      dtype='object')

In [19]:
data

,index,serviceCentre,dummy,overall rating,dummy1,geo_location,Unnamed: 5,Unnamed: 6,user,Unnamed: 8,date,rating,review,Unnamed: 12,response,review_original,topics,sentiment
0,0,Ford of Queens Service Center,Car Service Station,1.0,624,"92-15 170th St, Woodside, NY 11433, United States",\n+1 718-865-8591,https://www.fordofqueensboulevard.com/,Janghoon Jeon,https://www.google.com/maps/contrib/1049414359...,4 months ago,1 star,The service department encountered challenges ...,3.0,Thank you for taking the time to review our se...,The service department encountered challenges ...,"[[Service Diagnostics, Service Cost]]",Unknown
1,1,Ford of Queens Service Center,Car Service Station,3.5,624,"92-15 170th St, Woodside, NY 11433, United States",\n+1 718-865-8591,https://www.fordofqueensboulevard.com/,Andy Nisthalal,https://www.google.com/maps/contrib/1088727238...,a month ago,1 star,Lies and condescending attitudes from the serv...,NaN,"Andy, thank you for taking the time to review ...",Lies and condescending attitudes from the serv...,"[[Product Quality, Service Diagnostics]]",Unknown
2,2,Ford of Queens Service Center,Car Service Station,3.5,624,"92-15 170th St, Woodside, NY 11433, United States",\n+1 718-865-8591,https://www.fordofqueensboulevard.com/,Najai Najai,https://www.google.com/maps/contrib/1017533967...,a month ago,1 star,"I walked in, there was about 4 employees at th...",NaN,"Najai, we are sorry to see your negative revie...","I walked in, there was about 4 employees at th...","[[Staff, Facilities]]",Unknown
3,3,Ford of Queens Service Center,Car Service Station,3.5,624,"92-15 170th St, Woodside, NY 11433, United States",\n+1 718-865-8591,https://www.fordofqueensboulevard.com/,Ranjeet Singh,https://www.google.com/maps/contrib/1035747110...,a month ago,1 star,I had a very bad experience in that location. ...,NaN,"Ranjeet, we apologize if you have received any...",I had a very bad experience in that location. ...,"[[Staff, Facilities]]",Unknown
4,4,Ford of Queens Service Center,Car Service Station,3.5,624,"92-15 170th St, Woodside, NY 11433, United States",\n+1 718-865-8591,https://www.fordofqueensboulevard.com/,Paul Knight,https://www.google.com/maps/contrib/1083373622...,a month ago,1 star,Took my truck in to get the front brakes repla...,NaN,"Paul, we are sorry to see your negative review...",Took my truck in to get the front brakes repla...,"[[Brake,Product Quality]]",Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,1775,Tesla,Car repair and maintenance service,4.0,828,"530 NJ-17, Paramus, NJ 07652, United States",\n+1 201-225-2544,https://www.tesla.com/paramusroute17,bleichmanklein lawyers ny nj,https://www.google.com/maps/contrib/1173073817...,5 years ago,4 stars,Interesting,NaN,NaN,Interesting,[Not Classified],Unknown
1177,1776,Tesla,Car repair and maintenance service,4.0,828,"530 NJ-17, Paramus, NJ 07652, United States",\n+1 201-225-2544,https://www.tesla.com/paramusroute17,Süreyya Çağlar,https://www.google.com/maps/contrib/1172518932...,2 months ago,5 stars,A beautiful showroom where you can see the new...,NaN,NaN,A beautiful showroom where you can see the new...,"[[Not Classified, Not Classified]\n-]",[neutral]
1178,1777,Tesla,Car repair and maintenance service,4.0,828,"530 NJ-17, Paramus, NJ 07652, United States",\n+1 201-225-2544,https://www.tesla.com/paramusroute17,Manuel Delgadillo,https://www.google.com/maps/contrib/1004720626...,2 years ago,5 stars,I know very little about this place but they s...,NaN,NaN,I know very little about this place but they s...,[[Not Classified]],Unknown
1179,1779,Tesla,Car repair and maintenance service,4.0,828,"530 NJ-17, Paramus, NJ 07652, United States",\n+1 201-225-2544,https://www.tesla.com/paramusroute17,조강훈,https://www.google.com/maps/contrib/1069169425...,9 months ago,4 stars,Too busy,NaN,NaN,Too busy,[[Not Classified]],[negative]


In [20]:
data.to_csv("sample_output_llama_3.2.csv")

In [ ]:
data=pd.read_csv("/Users/santhanakrishnans/Downloads/sample_output-4.csv")

In [ ]:
data.columns

In [ ]:
data

In [ ]:
import re

def cleanse_topics(topics_column):
    cleansed_topics = []
    for topics in topics_column:
        # Replace "-" or "--" with ","
        topics = re.sub(r"--|-", ",", topics)
        # Remove unwanted words and characters
        topics = re.sub(r"(?i)\b(sentiment|neutral|positive|negative)\b", "", topics)
        topics = topics.replace("\n", "").replace("/n", "")
        # Remove extra commas and whitespace
        topics = re.sub(r",+", ",", topics).strip(", ")
        cleansed_topics.append(topics.strip())
    return cleansed_topics

# Apply the function to data["topics"]
data["topics"] = cleanse_topics(data["topics"])

data["topics"]


In [ ]:
data["topics"] = data["topics"].replace("'", "").replace(",,", "").replace(":", "").replace("\n", "")


In [ ]:
data["topics"].to_csv("/Users/santhanakrishnans/Downloads/rawtopics.csv")

In [ ]:
import pandas as pd

# Step 1: Extract unique topics
def get_unique_topics(topics_column):
    all_topics = []
    for topics in topics_column:
        if topics:  # Ensure the value is not empty
            all_topics.extend([topic.strip() for topic in topics.split(",")])
    return list(set(all_topics))

# Step 2: Create new columns for each topic
def add_topic_columns(data, unique_topics):
    for topic in unique_topics:
        data[topic] = data["topics"].apply(lambda x: 1 if topic in x.split(",") else 0)
    return data

# Generate the unique topics list
unique_topics = get_unique_topics(data["topics"])

# Add new columns for each topic
data = add_topic_columns(data, unique_topics)

# Display updated DataFrame
data


In [ ]:
data.columns

In [ ]:
data.to_csv("sample_output_w_topics.csv")

In [ ]:
# Include 'sentiment' column along with the columns to the right of it
right_columns = data.loc[:, data.columns[data.columns.get_loc("sentiment"):]]

# Melt the data so that the columns right of 'Sentiment' become rows
melted_data = right_columns.melt(id_vars=["sentiment"], var_name="Category", value_name="Value")

# Create a pivot table to summarize by 'Sentiment' and 'Category'
pivot_table = melted_data.pivot_table(index="sentiment", columns="Category", values="Value", aggfunc="sum")

# Transpose the table
transposed_table = pivot_table.T

# Display the transposed table
transposed_table


In [ ]:
data_topic=transposed_table


In [ ]:
data_topic.to_csv("/Users/santhanakrishnans/Downloads/sampletopics_updated.csv")